In [ ]:
# Объявляю нужные библиотеки
import pandas as pd # Библиотека для работы с фреймворками
import matplotlib.pyplot as plt # Библиотека для визуализации данных
import seaborn as sns # Библиотека для визуализации данных
import numpy as np # Библиотека для математических операций

In [ ]:
# Загрузка датасета из csv-файла
df = pd.read_csv('vpnDataset.csv', sep=',')
df